In [65]:
import pandas as pd
import pymysql as pymysql

debug = True

start_date = '2016-01-01'
end_date = '2020-01-01'
resample_interval = '5T'

host = "mariadb.mmto.arizona.edu"
database = "measurements"
table = 'bump'
usr = "webuser"
pw = "Clear7Skies!"

conn = pymysql.connect(host=host, port=3306, user=usr, passwd=pw, db=database)

In [66]:
#  This gets the individual e-series thermocouples parameter names
def get_names(connection):
    df = None
    try:
        # Read data
        with connection.cursor() as cursor:
            sql = f"""SELECT ds_name FROM aaa_parameters WHERE ds_name LIKE 'cell_e_series%'"""
            df = pd.read_sql(sql, conn)
    except Exception as e:
        print(f"Error: {e}")
    return df

In [67]:
def get_data(connection, param, database, start_date, end_date):
    df = None
    try:
        # Read data
        with connection.cursor() as cursor:
            sql = f"""SELECT from_unixtime(timestamp/1000) as ts, value as {param} FROM {param} 
                  WHERE timestamp >= UNIX_TIMESTAMP('{start_date}') * 1000
                  AND timestamp < UNIX_TIMESTAMP('{end_date}') * 1000;"""
            # Using a median filter to remove spikes.  Modify as you wish.
            df = pd.read_sql(sql, conn, index_col='ts').resample(resample_interval).median()
    except Exception as e:
        print(f"Error: {e}")
    return df

In [68]:
df_names = get_names(conn)
df_names

,ds_name
0,cell_e_series_backplate_C
1,cell_e_series_backplate_max_C
2,cell_e_series_backplate_min_C
3,cell_e_series_carrier_offset_C
4,cell_e_series_carrier_reference_C
5,cell_e_series_chamber_ambient_C
6,cell_e_series_count
7,cell_e_series_crate_status
8,cell_e_series_diff_C
9,cell_e_series_frontplate_C


In [69]:
drop_list = [
    "cell_e_series_update",
    "cell_e_series_unit_status",
    "cell_e_series_timestamp",
    "cell_e_series_count",
    "cell_e_series_crate_status",
    "cell_e_series_ijb_raw",
    "cell_e_series_interval",
    "cell_e_series_restarted",
    "cell_e_series_server_status"
]
par_list = [
    "cell_e_series_backplate_C",
    "cell_e_series_chamber_ambient_C",
    "cell_e_series_frontplate_C",
    "cell_e_series_in_front_of_primary_C",
    "cell_e_series_lower_plenum_C",
    "cell_e_series_midplate_C",
    "cell_e_series_outside_ambient_C",
    "yankee_temperature",
    "temptrax1_probe2",
    "temptrax1_probe3",
    "temptrax1_probe4",
    "temptrax1_probe6",
    "temptrax3_probe10",
    "temptrax3_probe11",
    "temptrax3_probe12"
]

In [70]:
df_all = None

for name in par_list:
    if debug:
        print(f"Getting {name} values...")
    df = get_data(conn, name, database, start_date, end_date)
    if df_all is None:
        df_all = df
    else:
        if df is not None:
            df_all = df_all.join(df, how='inner')

Getting cell_e_series_backplate_C values...
Getting cell_e_series_chamber_ambient_C values...
Getting cell_e_series_frontplate_C values...
Getting cell_e_series_in_front_of_primary_C values...
Getting cell_e_series_lower_plenum_C values...
Getting cell_e_series_midplate_C values...
Getting cell_e_series_outside_ambient_C values...
Getting yankee_temperature values...
Getting temptrax1_probe2 values...
Getting temptrax1_probe3 values...
Getting temptrax1_probe4 values...
Getting temptrax1_probe6 values...
Getting temptrax3_probe10 values...
Getting temptrax3_probe11 values...
Getting temptrax3_probe12 values...


In [71]:
df_all.tail()

,cell_e_series_backplate_C,cell_e_series_chamber_ambient_C,cell_e_series_frontplate_C,cell_e_series_in_front_of_primary_C,cell_e_series_lower_plenum_C,cell_e_series_midplate_C,cell_e_series_outside_ambient_C,yankee_temperature,temptrax1_probe2,temptrax1_probe3,temptrax1_probe4,temptrax1_probe6,temptrax3_probe10,temptrax3_probe11,temptrax3_probe12
ts,,,,,,,,,,,,,,,
2019-05-28 22:35:00,6.88495,6.67865,6.79635,6.59840,6.61425,6.77050,6.08515,6.1250,6.5,6.5,6.6,5.70,6.3,6.65,6.9
2019-05-28 22:40:00,6.83840,6.57740,6.75030,6.53610,6.57290,6.72380,5.95010,6.2735,6.4,6.5,6.6,5.75,6.3,6.60,6.9
2019-05-28 22:45:00,6.82020,6.64990,6.73530,6.53590,6.54240,6.70490,5.97060,6.2025,6.4,6.4,6.6,5.80,6.2,6.60,6.8
2019-05-28 22:50:00,6.76285,6.69625,6.68565,6.47075,6.49205,6.65235,5.93665,6.1305,6.3,6.4,6.6,5.80,6.2,6.40,6.8
2019-05-28 22:55:00,6.74455,6.73505,6.66975,6.45775,6.48380,6.63945,5.90550,6.2270,6.3,6.4,6.6,5.80,6.1,6.40,6.8


In [72]:
df_all.shape

(358260, 15)

In [73]:
df_all.to_csv("../halcoll/data/halcoll_temps.csv")

In [ ]:
# temptrax1 2, 3, 4, 6 (ne top, se top, se bottom, hub)
# temptrax3 10, 11, 12 (nw top, sw top, sw bottom)
# yankee_temperature